In [1]:
from fastai.vision import *
from CinemaNet.train_utils import *

### Load Model

In [3]:
data = get_data('/home/rahul/github_projects/CinemaNet/', base_size=224, batch_size=1)

In [4]:
learn = cnn_learner(data, models.mobilenet_v2)

In [5]:
!ls ../models

MobileNetV2-img_224-frozen-opt_accuracy.pth
MobileNetV2-img_224-frozen-opt_val_loss.pth
MobileNetV2-img_224-unfrozen-opt_accuracy.onnx
MobileNetV2-img_224-unfrozen-opt_accuracy.pth
MobileNetV2-img_224-unfrozen-opt_val_loss.pth
tmp.pth


### PyTorch ⟶ ONNX

In [6]:
fname = 'MobileNetV2-img_224-frozen-opt_accuracy'
learn.load(fname);

In [7]:
#export
def torch_to_onnx(model:nn.Module,
                  save_path   :str   = '../models/',
                  model_fname :str   = 'onnx-model',
                  img_size    :tuple = to_film_ar(224)):
    model.eval();
    x = torch.randn(1, 3, *img_size)
    torch.onnx._export(model, x.cuda(), 
                       f'{os.path.join(save_path, model_fname)}.onnx',
                       export_params=True)


In [8]:
torch_to_onnx(learn.model, model_fname=fname)

### ONNX ⟶ CoreML

In [9]:
imagenet_stats

([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [10]:
#export
import copy
import coremltools
import os

from onnx_coreml import convert
from onnx import onnx_pb

In [11]:
#export
red_scale = 1.0 / (0.229 * 255.0)
green_scale = 1.0 / (0.224 * 255.0)
blue_scale = 1.0 / (0.225 * 255.0)

args = dict(is_bgr=False, red_bias = -(0.485 * 255.0)  , green_bias = -(0.456 * 255.0)  , blue_bias = -(0.406 * 255.0))

In [12]:
#export
def onnx_to_coreml(path_to_model:str, 
                   num_labels:int,
                   mode:str = 'classifier',
                   preprocessing_args:dict = args,
                   target_ios:str = '13'):
    return convert(
        model=path_to_model, mode=mode, image_input_names=['input.1'], 
        class_labels=[i for i in range(num_labels)], preprocessing_args=args,
        target_ios=target_ios
    )

In [17]:
path_to_model = '../models/MobileNetV2-img_224-frozen-opt_accuracy.onnx'
mod_converted = onnx_to_coreml(path_to_model, 6, target_ios='13')

1/166: Converting Node Type Conv
2/166: Converting Node Type BatchNormalization
3/166: Converting Node Type Clip
4/166: Converting Node Type Conv
5/166: Converting Node Type BatchNormalization
6/166: Converting Node Type Clip
7/166: Converting Node Type Conv
8/166: Converting Node Type BatchNormalization
9/166: Converting Node Type Conv
10/166: Converting Node Type BatchNormalization
11/166: Converting Node Type Clip
12/166: Converting Node Type Conv
13/166: Converting Node Type BatchNormalization
14/166: Converting Node Type Clip
15/166: Converting Node Type Conv
16/166: Converting Node Type BatchNormalization
17/166: Converting Node Type Conv
18/166: Converting Node Type BatchNormalization
19/166: Converting Node Type Clip
20/166: Converting Node Type Conv
21/166: Converting Node Type BatchNormalization
22/166: Converting Node Type Clip
23/166: Converting Node Type Conv
24/166: Converting Node Type BatchNormalization
25/166: Converting Node Type Add
26/166: Converting Node Type Conv


TypeError: Error while converting op of type: BatchNormalization. Error message: provided number axes 0 not supported 
 Please try converting with higher target_ios.
You can also provide custom function/layer to convert the model.


In [ ]:
(list(learn.model.children()))

[Sequential(
   (0): Sequential(
     (0): ConvBNReLU(
       (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU6(inplace=True)
     )
     (1): InvertedResidual(
       (conv): Sequential(
         (0): ConvBNReLU(
           (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
           (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): ReLU6(inplace=True)
         )
         (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
     )
     (2): InvertedResidual(
       (conv): Sequential(
         (0): ConvBNReLU(
           (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
           (1): BatchNorm2d(96, eps=1e-05, momentum